## Code for automatic merging of Mozambique data

Some code to process XLS data and merge it into an appropriate format.

**After longer work on this - retreat! - do manual merging**

In [122]:
import os
import xlrd
import numpy as np
import pandas as pd
from termcolor import colored
import unidecode
import difflib

In [123]:
file_path = "C://Users//wb555300//OneDrive - WBG//Municipality_merge//Source_tables"
os.chdir(file_path)
os.getcwd()

'C:\\Users\\wb555300\\OneDrive - WBG\\Municipality_merge\\Source_tables'

#### Functions:

In [124]:
#Create a function to check if format is xls:
def is_xls(files):
    xls_format = []
    for x in files:
        leng = len(x.strip().split("."))
        xls_format.append(x.strip().split(".")[leng-1]=="xls")
    return np.array(xls_format)


# split at multiple different seperators:
def multi_split(txt, seps):
    default_sep = seps[0]

    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]


## Functions to identify non-empty rows and columns:

def nonempty_rows(sheet):
    empty_row = []
    for row in range(sheet.nrows):
        empty_row.append(all(np.array(sheet.row_values(row,0))==''))
    return np.where(np.array(empty_row)==False)[0]

def nonempty_cols(sheet):
    empty_col = []
    for col in range(sheet.ncols):
        empty_col.append(all(np.array(sheet.col_values(col,0))==''))
    return np.where(np.array(empty_col)==False)[0]



#### Code:

In [125]:
#Make list with all files in directory
all_files = np.array(os.listdir())
xls_files = all_files[is_xls(all_files)]
print(xls_files)

['BEIRA_ Informação Financeira Municipal 2013 a 2019.xls'
 'CHIBUTO  Informação Financeira Municipal 2013 a 2019.xls'
 'CHIMOIO_ Informação Financeira Municipal 2013 a 2019.xls'
 'DONDO_ Financeira Municipal 2013 a 2019.xls'
 'GONDOLA Informação Financeira Municipal 2013 a 2019.xls'
 'GORONGOSA_Informação Financeira Municipal 2013 a 2019.xls'
 'GURUE_RECEITA 2013-2019.xls'
 'LICHINGA_Colecta de  Informação Financeira Municipal 2013 a JUNHO 2019.xls'
 'MALEMA_EXECUCAO FINANCEIRA DA AUTARQUIA 2013-2019.xls'
 'MANDIMBA_Info.xls'
 'MANDLAKAZI_ Informação Financeira Municipal 2013 a 2019.xls'
 'MAPUTO Evolução da Execução Orçamental 2013 - 2019.xls'
 'MARROMEU_ Informação Financeira Municipal 2013 a 2019.xls'
 'MASSINGA_  InformaÃ§Ã£o Financeira Municipal 2013 a JUNHO 2019.xls'
 'MATOLA_ Informação Financeira Municipal 2013 a 2019.xls'
 'MAXIXE  Informação Financeira Municipal 2013 a 2019.xls'
 'MOCIMBOA DA PRAIA_COLECTA DE INFOR. 2013-2019.xls'
 'MOCUBA_Informação Financeira Municipal 2013

In [126]:
xls_files[0:2]

array(['BEIRA_ Informação Financeira Municipal 2013 a 2019.xls',
       'CHIBUTO  Informação Financeira Municipal 2013 a 2019.xls'],
      dtype='<U74')

### Extract all tables from one file:

Extract current municipality name:

In [130]:
out_loc = 'C:/Users/wb555300/OneDrive - WBG/Municipality_merge/Version2/'

## Save list with problematic files
problem_files = []

## List with municipalities:
munis = []
# and available municipalities
munis_avail = []


for file in xls_files:

    #Extract muni name:
    muni_name = multi_split(file, seps = [" ", "_", "."])[0]
    munis.append(muni_name)
    print('Currently processing data on: ', muni_name)
    
    #Open file:
    wb = xlrd.open_workbook(file)
    sheet = wb.sheet_by_index(0) 

    # Extract number of rows and columns in file:
    ne_rows = nonempty_rows(sheet)
    ne_cols = nonempty_cols(sheet)

    # identify rows and columns where table starts (all tables tart with DESCRICAO):
    start_rowcol = []

    for row in ne_rows:
        for col in ne_cols:
            if(sheet.cell_value(row, col)=="DESCRICAO"): 
                start_rowcol.append([row, col])


    #Extract all tables:
    table_count = 1

    for c_start in start_rowcol:

        year_row = c_start[0]
        OE_row = c_start[0]+1

        years = sheet.row_values(year_row, c_start[1]+1)
        OEs = sheet.row_values(OE_row, c_start[1]+1)
        OEs = list(map(lambda x: unidecode.unidecode(x), OEs))

        # duplicate year vector:
        for x in range(len(years)):
            if years[x] == '':
                years[x] = years[x-1]

        i = 2
        current_rowname_start = multi_split(sheet.cell_value(c_start[0]+i, c_start[1]), [" ", "_", "."])[0]

        temp_df = pd.DataFrame({'muni': np.repeat(muni_name, len(years)),'year': np.array(years), 'OE': np.array(OEs)})
        temp_df_cols = 3

        while current_rowname_start != "TOTAL":
            current_rowname_start = multi_split(sheet.cell_value(c_start[0]+i, c_start[1]), [" ", "_", "."])[0]
            if sheet.cell_value(c_start[0]+i, c_start[1]-1)!='':
                temp_num = sheet.cell_value(c_start[0]+i, c_start[1]-1)
            temp_name = str(temp_num).replace('.','_')+'-'+unidecode.unidecode(sheet.cell_value(c_start[0]+i, c_start[1])).upper().strip().replace("  "," ").replace(" ", "_").replace(",", "_")
            temp_values = sheet.row_values(c_start[0]+i, c_start[1]+1)
                      
            temp_df.insert(temp_df_cols, temp_name, temp_values, True)

            temp_df_cols = temp_df_cols +1
            i = i+1

        csv_file_path = out_loc + muni_name + '_Table_' + str(table_count) +'.csv'
    
        temp_df.to_csv(csv_file_path)
        table_count = table_count + 1
     
    if table_count-1 != 3: 
        print(colored(str(table_count-1) + ' tables found \n', color='red'))
        problem_files.append(file)
    else:     
        print(colored(str(table_count-1) + ' tables found \n'))
        munis_avail.append(muni_name)
        
    

Currently processing data on:  BEIRA
3 tables found 

Currently processing data on:  CHIBUTO
3 tables found 

Currently processing data on:  CHIMOIO
3 tables found 

Currently processing data on:  DONDO
3 tables found 

Currently processing data on:  GONDOLA
3 tables found 

Currently processing data on:  GORONGOSA
3 tables found 

Currently processing data on:  GURUE
0 tables found 

Currently processing data on:  LICHINGA
3 tables found 

Currently processing data on:  MALEMA
3 tables found 

Currently processing data on:  MANDIMBA
4 tables found 

Currently processing data on:  MANDLAKAZI
3 tables found 

Currently processing data on:  MAPUTO
0 tables found 

Currently processing data on:  MARROMEU
3 tables found 

Currently processing data on:  MASSINGA
3 tables found 

Currently processing data on:  MATOLA
3 tables found 

Currently processing data on:  MAXIXE
3 tables found 

Currently processing data on:  MOCIMBOA
3 tables found 

Currently processing data on:  MOCUBA
3 tables f

In [131]:
print("Problem files: ", [multi_split(file, seps = [" ", "_", "."])[0] for file in problem_files] ,"\n")

Problem files:  ['GURUE', 'MANDIMBA', 'MAPUTO', 'MOLOCUE'] 



In [196]:
# Code to open a respective table:
def open_table(muni, table, path):
    path_loc = path + muni + '_Table_' + str(table) +'.csv'
    return pd.read_csv(path_loc, index_col=0)

### Simply merge tables:
In the following all tables are merged independently of similar column names. The code to replace similar column names can be found below... It was not used in the end as there were too many similarities in column names that needs to be judged on a one by one basis.

#### Table 1

In [213]:
overview_table_1 = open_table(munis_avail[0],1, path = out_loc)

for muni in munis_avail:
    print("processing " + muni)
    temp_table = open_table(muni,1, out_loc).iloc[0:14]
    #temp_table = trans_similar_columns(overview_table_1, temp_table)
    overview_table_1 = overview_table_1.append(temp_table, sort = False)
    print("\n")

overview_table_1.to_csv("../Overview/Table_1/Overview_Table_1.csv")

processing BEIRA


processing CHIBUTO


processing CHIMOIO


processing DONDO


processing GONDOLA


processing GORONGOSA


processing LICHINGA


processing MALEMA


processing MANDLAKAZI


processing MARROMEU


processing MASSINGA


processing MATOLA


processing MAXIXE


processing MOCIMBOA


processing MOCUBA


processing MONAPO


processing MUEDA


processing NACALA


processing NHAMATANDA


processing QUELIMANE


processing QUISSICO1


processing TETE


processing VILANKULO




#### Table 2

In [214]:
overview_table_2 = open_table(munis_avail[0],2, path = out_loc)

for muni in munis_avail[1:]:
    print("processing " + muni)
    temp_table = open_table(muni,2, out_loc).iloc[0:14]
    #temp_table = trans_similar_columns(overview_table_1, temp_table)
    overview_table_2 = overview_table_2.append(temp_table, sort = False)
    print("\n")

overview_table_2.to_csv("../Overview/Table_2/Overview_Table_2.csv")

processing CHIBUTO


processing CHIMOIO


processing DONDO


processing GONDOLA


processing GORONGOSA


processing LICHINGA


processing MALEMA


processing MANDLAKAZI


processing MARROMEU


processing MASSINGA


processing MATOLA


processing MAXIXE


processing MOCIMBOA


processing MOCUBA


processing MONAPO


processing MUEDA


processing NACALA


processing NHAMATANDA


processing QUELIMANE


processing QUISSICO1


processing TETE


processing VILANKULO




#### Table 3

In [215]:
overview_table_3 = open_table(munis_avail[0],3, path = out_loc)

for muni in munis_avail[1:]:
    print("processing " + muni)
    temp_table = open_table(muni,3, out_loc).iloc[0:14]
    #temp_table = trans_similar_columns(overview_table_3, temp_table)
    overview_table_3 = overview_table_3.append(temp_table, sort = False)
    print("\n")

overview_table_3.to_csv("../Overview/Table_3/Overview_Table_3.csv")

processing CHIBUTO


processing CHIMOIO


processing DONDO


processing GONDOLA


processing GORONGOSA


processing LICHINGA


processing MALEMA


processing MANDLAKAZI


processing MARROMEU


processing MASSINGA


processing MATOLA


processing MAXIXE


processing MOCIMBOA


processing MOCUBA


processing MONAPO


processing MUEDA


processing NACALA


processing NHAMATANDA


processing QUELIMANE


processing QUISSICO1


processing TETE


processing VILANKULO




#### More automation by automatic column name replacement:

Function to replace similar column names - only if initial identifier agrees!

In [199]:
def trans_similar_columns(df1, df2, threshold = 0.8, trace = True):
    ### Function compares column names of df2 with the names of df1 and replaces 
    ### names in df2 with the names of df1 if they are sufficiently similar!

    col1 = list(df1.columns)[3:]
    col2 = list(df2.columns)[3:]

    col2_new = col2.copy()

    for i2 in range(len(col2)):
        #Iterate through columns of the existing data frame:

        indicator1 = np.array([(col2_new[i2].split("-")[0] == col1[i1].split("-")[0]) for i1 in range(len(col1))])
        indicator2 = np.array([difflib.SequenceMatcher(lambda z: z == " ", (col2_new[i2].split("-"))[1],col1[i1].split("-")[1]).ratio() for i1 in range(len(col1))])        
        indicator_t = indicator1*indicator2 > threshold 

        if any(indicator_t):
            indi = int(np.where(indicator2 == indicator2.max())[0][0])
            col2_new[i2] = col1[indi]
            if indicator2.max() != 1 and trace == True:
                print(col2_new[i2], 'was replaced with', col1[indi])
        else:
            temp = np.where(indicator2 == indicator2.max())[0][0]
            indi = int(temp)
            if trace == True:
                print(colored(str(col2_new[i2])+' has highest similarity of '+ str(indicator2.max().round(2)) +' with ' + str(col1[indi]), color = "red"))

    df2.columns = list(df2.columns)[:3]+col2_new
    return df2

In [207]:
## Merge all Data Frames

overview_table_1 = open_table(munis_avail[0],1, path = out_loc)

for muni in munis_avail:
    print("processing " + muni)
    temp_table = open_table(muni,1, out_loc).iloc[0:14]
    temp_table = trans_similar_columns(overview_table_1, temp_table)
    overview_table_1 = overview_table_1.append(temp_table, sort = False)
    print("\n")

overview_table_1.to_csv("../Overview/Table_1/Overview_Table_1.csv")

processing BEIRA


processing CHIBUTO
2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU) was replaced with 2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU)
2_1-FRA has highest similarity of 0.67 with 2_1-FCA
2_2-OUTROS has highest similarity of 0.53 with 2_2-PROMAPUTO


processing CHIMOIO
2_1-PERPU has highest similarity of 0.57 with 2_2-PROMAPUTO
2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU) was replaced with 2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU)
2_1-PERPU_REMBOLSO has highest similarity of 0.43 with 2_2-PROMAPUTO
2_1-OUTROS has highest similarity of 1.0 with 2_2-OUTROS
2_2-FUNDO_DE_COOP._CONJUNTA has highest similarity of 0.56 with 2_1-FUNDO_CAPITAL
2_2-FUNDO_UN_HABITAT has highest similarity of 0.69 with 2_1-FUNDO_CAPITAL
2_2-FUSP has highest similarity of 0.4 with 2_2-OUTROS
2_2-PRODIA has highest similarity of 0.67 with 2_2-PDA


processing DONDO
2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU) was replaced with 2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU)
2_2-OUTROS_(RESPONSABILIDADE_SOCIAL;_IDAI) h

ValueError: Plan shapes are not aligned

In [212]:
#overview_table_1.append(temp_table)
temp_table.columns.duplicated()
temp_table.columns

Index(['muni', 'year', 'OE', '1_0-RECEITAS_PROPRIAS', '1_0-RECEITAS_FISCAIS',
       '1_0-RECEITAS_NAO_FISCAIS', '1_0-RENDIMENTO_DE_BENS_MOVEIS_E_IMOVEIS',
       '2_0-RECEITAS_EXTERNAS_(_2_1+2_2)', '2_1-TRANSFERENCIAS_DO_ESTADO',
       '2_1-FCA', '2_1-FIIL', '2_1-FUNDOS_DE_ESTRADAS', '2_1-FUNDO_CAPITAL',
       '2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU)', '2_1-FRA',
       '2_2-FINANCIAMENTO_EXTERNO_-_(CREDITOS+DONATIVOS+EMPRESBANCARIOS)',
       '2_2-PROMAPUTO', '2_2-OUTROS', '2_2-PCMC', '2_2-COOP._ESPANHOLA',
       '2_2-ORCAMENTACAO_PARTICIPATIVA', '2_2-P13', '2_2-PDA', '2_2-PRODEM',
       '2_2-UNIAO_EUROPEIA', '2_2-PRODEM',
       '3_0-TOTAL_DE_RECEITAS_PROPRIAS_(1+2)'],
      dtype='object')

In [177]:
df1 = open_table(munis_avail[0],1)
df2 = open_table(munis_avail[2],1)
trace = True

col1 = list(df1.columns)[3:]
col2 = list(df2.columns)[3:]

col2_new = col2.copy()
    
for i2 in range(len(col2)):
    #Iterate through columns of the existing data frame:
    
    indicator1 = np.array([(col2_new[i2].split("-")[0] == col1[i1].split("-")[0]) for i1 in range(len(col1))])
    indicator2 = np.array([difflib.SequenceMatcher(lambda z: z == " ", (col2_new[i2].split("-"))[1],col1[i1].split("-")[1]).ratio() for i1 in range(len(col1))])        
    indicator_t = indicator1*indicator2 > threshold 
    
    if any(indicator_t):
        indi = int(np.where(indicator2 == indicator2.max())[0])
        col2_new[i2] = col1[indi]
        if indicator2.max() != 1 and trace == True:
            print(col2_new[i2], 'was replaced with', col1[indi])
    else:
        temp = np.where(indicator2 == indicator2.max())[0][0]
        indi = int(temp)
        if trace == True:
            print(colored(str(col2_new[i2])+' has highest similarity of '+ str(indicator2.max().round(2)) +' with ' + str(col1[indi]), color = "red"))

df2.columns = list(df2.columns)[:3]+col2_new

2_1-PERPU has highest similarity of 0.57 with 2_2-PROMAPUTO
2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU) was replaced with 2_1-OUTRAS_TRANSFERENCIAS_DO_ESTADO_(FRPU)
2_1-PERPU_REMBOLSO has highest similarity of 0.43 with 2_2-PROMAPUTO
2_1-OUTROS has highest similarity of 0.53 with 2_2-PROMAPUTO
2_2-OUTROS has highest similarity of 0.53 with 2_2-PROMAPUTO
2_2-FUNDO_DE_COOP._CONJUNTA has highest similarity of 0.56 with 2_1-FUNDO_CAPITAL
2_2-FUNDO_UN_HABITAT has highest similarity of 0.69 with 2_1-FUNDO_CAPITAL
2_2-FUSP has highest similarity of 0.35 with 2_1-FUNDO_CAPITAL
2_2-PRODIA has highest similarity of 0.67 with 2_2-PDA


In [12]:
overview_table_2 = pd.DataFrame()

for muni in munis_avail:
    temp_table = open_table(muni,2)
    overview_table_2 = overview_table_2.append(temp_table, sort = False)

overview_table_2.to_csv("../Overview/Overview_Table_2.csv")

In [103]:
df1 = open_table(munis[0],1)
df2 = open_table(munis[4],1)

trans_similar_columns(df1,df2)

OUTRAS TRANSFERENCIAS DO ESTADO (FRPU) was replaced with OUTRAS TRANSFERENCIAS DO ESTADO (FRPU)
FRA has highest similarity of 0.67 with FCA
PAPELINE has highest similarity of 0.36 with PDA
GIZ has highest similarity of 0.29 with FIIL
MAE has highest similarity of 0.4 with OE
OSEO has highest similarity of 0.67 with OE
GAPI has highest similarity of 0.35 with FUNDO CAPITAL


,muni,year,OE,RECEITAS PROPRIAS,RECEITAS FISCAIS,RECEITAS NAO FISCAIS,RENDIMENTO DE BENS MOVEIS E IMOVEIS,"RECEITAS EXTERNAS ( 2,1+2,2)",TRANSFERENCIAS DO ESTADO,FCA,...,PROMAPUTO,PAPELINE,PCMC,GIZ,MAE,OSEO,GAPI,PRODEM,UNIAO EUROPEIA,TOTAL DE RECEITAS PROPRIAS (1+2)
0,GONDOLA,2013.0,ORCAMENTO,4200000.00,825000.00,3375000.00,0.00,29736220.00,23186220.00,11121730.00,...,0.0,303000.0,1660000.0,0.00,4300000.00,287000.0,0.0,0.0,0.0,33936220.00
1,GONDOLA,2013.0,EXECUCAO,2557141.32,213284.00,2343857.32,0.00,34336006.57,27974672.80,11121729.80,...,0.0,303000.0,1663050.0,96800.00,4298483.77,0.0,0.0,0.0,0.0,36893147.89
2,GONDOLA,2014.0,ORCAMENTO,4204196.72,1111196.72,3093000.00,0.00,29914143.88,27463463.88,13715075.18,...,0.0,312000.0,1888680.0,0.00,0.00,250000.0,0.0,0.0,0.0,34118340.60
3,GONDOLA,2014.0,EXECUCAO,2924975.84,186445.50,2738530.34,0.00,29083786.57,26880482.01,13720280.31,...,0.0,312000.0,1888680.0,2624.56,0.00,0.0,0.0,0.0,0.0,32008762.41
4,GONDOLA,2015.0,ORCAMENTO,5563800.99,1562866.62,4000934.37,0.00,30469890.00,27992710.00,13792620.00,...,0.0,345000.0,1888680.0,0.00,0.00,243500.0,0.0,0.0,0.0,36033690.99
5,GONDOLA,2015.0,EXECUCAO,5372715.75,597952.00,4774763.75,0.00,35727065.29,28476225.29,13825784.29,...,0.0,345000.0,1888680.0,17160.00,5000000.00,0.0,0.0,0.0,0.0,41099781.04
6,GONDOLA,2016.0,ORCAMENTO,7389310.24,857333.22,6531977.02,0.00,35165339.80,31965050.00,14979030.00,...,0.0,350000.0,2850289.8,0.00,0.00,0.0,0.0,0.0,0.0,42554650.04
7,GONDOLA,2016.0,EXECUCAO,5829046.70,909564.10,4919482.60,0.00,33398491.45,29795504.25,14979030.64,...,0.0,570000.0,2850290.0,58867.20,0.00,123830.0,0.0,0.0,0.0,39227538.15
8,GONDOLA,2017.0,ORCAMENTO,7389310.24,1157333.22,6231977.02,0.00,36878267.00,26884550.00,16130730.00,...,0.0,598000.0,3895717.0,0.00,5500000.00,0.0,0.0,0.0,0.0,44267577.24
9,GONDOLA,2017.0,EXECUCAO,6426804.50,1057617.45,5369187.05,0.00,34052268.00,29558550.00,16130730.00,...,0.0,598000.0,3895718.0,0.00,0.00,0.0,0.0,0.0,0.0,40479072.50


In [145]:
overview_table_1 = pd.DataFrame()
overview_table_1 = overview_table_1.append(open_table("BEIRA",1))
overview_table_1

,muni,year,OE,RECEITAS PROPRIAS,RECEITAS FISCAIS,RECEITAS NAO FISCAIS,RENDIMENTO DE BENS MOVEIS E IMOVEIS,"RECEITAS EXTERNAS ( 2,1+2,2)",TRANSFERENCIAS DO ESTADO,FCA,...,PROMAPUTO,OUTROS (DONATIVOS),PCMC,COOP. ESPANHOLA,ORCAMENTACAO PARTICIPATIVA,P13,PDA,PRODEM,UNIAO EUROPEIA,TOTAL DE RECEITAS PROPRIAS (1+2)
0,BEIRA,2013.0,ORÇAMENTO,3.680504e+08,1.037888e+08,2.626616e+08,1600000.0,3.880842e+08,2.523196e+08,1.477978e+08,...,0.0,1.357646e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.561346e+08
1,BEIRA,2013.0,EXECUÇÃO,2.496427e+08,6.587861e+07,1.824772e+08,1286900.0,2.734374e+08,2.523197e+08,1.477980e+07,...,0.0,2.111774e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.230801e+08
2,BEIRA,2014.0,ORÇAMENTO,4.139027e+08,1.153200e+08,2.968707e+08,1712000.0,3.506254e+08,2.888420e+08,1.744014e+08,...,0.0,6.178341e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.645282e+08
3,BEIRA,2014.0,EXECUÇÃO,2.636741e+08,8.927405e+07,1.729861e+08,1413900.0,3.177486e+08,2.747058e+08,1.822610e+08,...,0.0,4.304287e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.814227e+08
4,BEIRA,2015.0,ORÇAMENTO,4.507108e+08,1.334190e+08,3.154086e+08,1883200.0,3.389403e+08,3.167569e+08,1.832915e+08,...,0.0,2.218341e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.896511e+08
5,BEIRA,2015.0,EXECUÇÃO,3.187132e+08,1.038807e+08,2.130947e+08,1737700.0,3.003485e+08,3.003485e+08,1.832915e+08,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.190616e+08
6,BEIRA,2016.0,ORÇAMENTO,4.234907e+08,1.226465e+08,2.986442e+08,2200000.0,3.965245e+08,3.436483e+08,1.990578e+08,...,0.0,5.287616e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.200152e+08
7,BEIRA,2016.0,EXECUÇÃO,3.206264e+08,6.704294e+07,2.513185e+08,2265000.0,3.351027e+08,3.202398e+08,1.990787e+08,...,0.0,1.486282e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.557291e+08
8,BEIRA,2017.0,ORÇAMENTO,4.899419e+08,4.438343e+08,4.335761e+07,2750000.0,4.617373e+08,3.821861e+08,2.143628e+08,...,0.0,7.955120e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.516793e+08
9,BEIRA,2017.0,EXECUÇÃO,3.214281e+08,2.921811e+08,2.663360e+07,2613400.0,3.819130e+08,3.293462e+08,2.143628e+08,...,0.0,5.256677e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.033410e+08


In [29]:
sheet.nrows
sheet.ncols

#find column and line 

for x in range(sheet.nrows):
    print(sheet.cell_value(x,0),sheet.cell_value(x,1),sheet.cell_value(x,2))

  
  
  
 EVOLUÇÃO DA EXECUÇÃO ORÇAMENTAL DE RECEITAS E DESPESAS DO CMC - CHIBUTO 
  
  
 DESCRICAO 2013.0
  ORÇAMENTO
1.0 RECEITAS PROPRIAS 15338210.0
 RECEITAS FISCAIS 4797780.0
 RECEITAS NAO FISCAIS 10540430.0
 RENDIMENTO DE BENS MOVEIS E IMOVEIS 0.0
2.0 RECEITAS EXTERNAS ( 2,1+2,2) 58445140.0
2.1 TRANSFERENCIAS DO ESTADO 55286140.0
 FCA 23935571.67
 FIIL 10638790.0
 FUNDOS DE ESTRADAS 7705000.0
 FUNDO CAPITAL 0.0
 OUTRAS TRANSFERENCIAS DO ESTADO 13006778.33
 FRA 0.0
2.2 FINANCIAMENTO EXTERNO - (CREDITOS+DONATIVOS+EMPRESBANCARIOS) 3159000.0
 PROMAPUTO 0.0
 OUTROS 0.0
 PCMC 3159000.0
 COOP.  ESPANHOLA 0.0
 ORCAMENTACAO PARTICIPATIVA 0.0
 P13 0.0
 PDA 0.0
 PRODEM 0.0
 UNIAO EUROPEIA 0.0
3.0 TOTAL DE RECEITAS PROPRIAS (1+2) 73783350.0
 DESPESAS 
 DESCRICAO 2013.0
  ORÇAMENTO
1.0 Deespesas Correntes 52085360.0
 Despesas com Pessoal 33577570.0
 Bens e Servicos 17314790.0
 Outras Despesas Correntes 1193000.0
2.0 Despesas de Capital 21697990.0
 Investimento Fixo 21697990.0
3.0 TOTAL DE DES